In [ ]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from datetime import datetime, timedelta
import sys
import matplotlib.pyplot as plt

## For SDK
from getpass import getpass
from odp_sdk import ODPClient

import cmocean

from odp_sdk.utils.numeric import *
from odp_sdk.utils.visual import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_palette(sns.color_palette("bright"))
import warnings
warnings.filterwarnings("ignore")

# Connect to API

In [ ]:
client = ODPClient(api_key=getpass(prompt='Insert your personal ODP API key:'), 
                       project="odp", client_name="odp")

# Pull data for desired lat/lon and timespan
Don't forget to specify if you want flagged data included or excluded

In [ ]:
df=client.casts(longitude=[0,15],
                latitude=[50,65], 
                timespan=['2018-06-01','2018-08-31'],
                include_flagged_data = True,
                n_threads=35) 

In [ ]:
df.head()

# Use built-in functions to get a sense of the data present

In [ ]:
missing_values(df, var_list = ['Oxygen', 'Temperature', 'Salinity', 'Chlorophyll','pH', 'Nitrate'])

In [ ]:
plot_nulls(df, var_list = ['Oxygen', 'Temperature', 'Salinity', 'Chlorophyll', 'pH', 'Nitrate'])

In [ ]:
plot_distributions(df, var_list = ['z'])

# Get units for each variable

In [ ]:
get_units()

# Plot casts for specific variable
Here we do Temperature and Oxygen. You can specify the colormap you want to use

In [ ]:
## available colormaps through cmocean
cmocean.cm.cmapnames

In [ ]:
plot_casts('Temperature',df,cmap=cmocean.cm.thermal,vrange=[0,20])

In [ ]:
plot_casts('Oxygen',df,cmap=cmocean.cm.oxy,vrange=[0,20]);plt.title('Ocean Temperatures');


# Let's take a look at some metadata
Get casts from 2018 in Norway region

In [ ]:
casts=client.get_available_casts(longitude=[-10,35],
                                latitude=[50,80],
                                timespan=['2018-06-01','2018-08-31'],
                                meta_parameters=['extId','date','time','lon','lat','country',
                                                 'Platform','dataset_code', 'equipment'])

casts.head()

In [ ]:
casts.head()

# Plot modes of data collection for each cast

In [ ]:
plot_datasets(casts, variable='equipment', longitude=[0,15],latitude=[50,65])

In [ ]:
plot_datasets(casts, variable='country', longitude=[0,15],latitude=[50,65])